In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.options.mode.chained_assignment = None

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-science-salaries-2023/ds_salaries.csv


# 1.Get an understanding of the dataset
1. Inspect the data
1. Check for null values
1. Check data types
1. Get some summary statistics

In [1]:
df = pd.read_csv('jobs_in_data.csv')


NameError: name 'pd' is not defined

In [ ]:
#1.1 - quick view of the data
df.head()

In [ ]:
#1.2 - Check the data types and see if are null values
df.info()

In [ ]:
df['company_location'].unique()

In [ ]:
#1.2 - Check for any null values
df.isnull().sum()

In [ ]:
#1.3 - Check data types 
df.dtypes

In [ ]:
#1.4 - Some summary statistics fot the 'salary' and 'salary_in_usd columns'
df[['salary','salary_in_usd']].describe()

# 2. Data Filtering and Cleaning

1. Replace values of vague columns and rename columns
1. Check for outliers and remove if necessary.
1. use a function to categorize the job_title values into a new column
1. create some vizualisaions to undertsand the data

In [ ]:
#2.1 - Make 'experince_level column more readable
df['experience_level'].replace(['SE', 'EX', 'EN', 'MI'],['Senior', 'Executive','Entry-level', 'Mid-level'], inplace=True)

In [ ]:
df

In [9]:
#2.1 - replace country initials
def replace_country_initials(df):
    # Updated mapping of country initials to full names
    country_mapping = {
        'DE': 'Germany', 'US': 'United States', 'GB': 'United Kingdom', 'CA': 'Canada',
        'ES': 'Spain', 'IE': 'Ireland', 'ZA': 'South Africa', 'PL': 'Poland',
        'FR': 'France', 'NL': 'Netherlands', 'LU': 'Luxembourg', 'LT': 'Lithuania',
        'PT': 'Portugal', 'GI': 'Gibraltar', 'AU': 'Australia', 'CO': 'Colombia',
        'UA': 'Ukraine', 'SI': 'Slovenia', 'IN': 'India', 'RO': 'Romania',
        'GR': 'Greece', 'LV': 'Latvia', 'MU': 'Mauritius', 'RU': 'Russia',
        'IT': 'Italy', 'KR': 'South Korea', 'EE': 'Estonia', 'CZ': 'Czech Republic',
        'CH': 'Switzerland', 'BR': 'Brazil', 'QA': 'Qatar', 'KE': 'Kenya',
        'DK': 'Denmark', 'GH': 'Ghana', 'SE': 'Sweden', 'PH': 'Philippines',
        'TR': 'Turkey', 'AD': 'Andorra', 'EC': 'Ecuador', 'MX': 'Mexico',
        'IL': 'Israel', 'NG': 'Nigeria', 'SA': 'Saudi Arabia', 'NO': 'Norway',
        'AR': 'Argentina', 'JP': 'Japan', 'HK': 'Hong Kong', 'CF': 'Central African Republic',
        'FI': 'Finland', 'SG': 'Singapore', 'TH': 'Thailand', 'HR': 'Croatia',
        'AM': 'Armenia', 'BA': 'Bosnia and Herzegovina', 'PK': 'Pakistan',
        'IR': 'Iran', 'BS': 'Bahamas', 'HU': 'Hungary', 'AT': 'Austria',
        'PR': 'Puerto Rico', 'AS': 'American Samoa', 'BE': 'Belgium', 'ID': 'Indonesia',
        'EG': 'Egypt', 'AE': 'United Arab Emirates', 'MY': 'Malaysia',
        'HN': 'Honduras', 'DZ': 'Algeria', 'IQ': 'Iraq', 'CN': 'China',
        'NZ': 'New Zealand', 'CL': 'Chile', 'MD': 'Moldova', 'MT': 'Malta'
    }
    
    # Replace the initials with full country names
    df['company_location'] = df['company_location'].map(country_mapping).fillna(df['company_location'])
    return df

In [10]:
df = replace_country_initials(df)

NameError: name 'df' is not defined

In [ ]:
# 2.1 - check if function worked
df['company_location']

In [ ]:
#2.1 - Make 'employment_type' column more readable
df['employment_type'].replace(['PT', 'FT', 'CT', 'FL'],['Part-time', 'Full-time', 'Contract', 'Freelance'], inplace=True)

In [ ]:
#2.1 - Check values of the `remote_ratio` columns
df['remote_ratio'].unique()

In [ ]:
#2.1 - Its values aren't very descriptive. we'll replace them to make them more readable.
df.rename(columns={'remote_ratio': 'work_setting'}, inplace=True)
df['work_setting'].replace([100, 50, 0],['Remote', 'Hybrid', 'In-person'], inplace=True)

In [ ]:
#2.2 - Use a boxplot to look for outliers
plt.figure(figsize=(12, 1))
sns.boxplot(x=df['salary_in_usd'])
plt.title('Boxplot of Salary in USD');

In [ ]:
#2.2 - Check outliers
df['salary_in_usd'].sort_values(ascending=False)

In [ ]:
#2.2 - Filter out the outliers
df = df[df['salary'] <= 450000]

In [ ]:
#2.2 - Check the filter
df['salary'].sort_values(ascending=False)

In [ ]:
# 2.3 - check how many unique job titles we have in the dataset
df['job_title'].unique()

Currently we have 105 unqiue job titles. Though this is good for detail, it is very difficult to vizualize and dril down. We need to categorize these to make them better suited for analysis. We can create a function that will parse the data and add a new column with the job titles categorized.

In [ ]:
# 2.3 - create a function that will categorize the job titles into a new column
def group_job_titles(job_title):
    # Define keywords for each specific category
    core_data_analysis_keywords = ['Data Analyst', 'Financial', 'Business Data', 'Compliance', 'Product Data']
    advanced_data_science_keywords = ['Data Scientist', 'Research Scientist', 'Research Analyst', 'Applied Scientist', 'Decision Scientist']
    ml_development_keywords = ['Machine Learning Researcher', 'Machine Learning Engineer', 'Deep Learning', 'NLP', 'Computer Vision Engineer', 'ML Scientist', 'Applied ML']
    ai_specialization_keywords = ['AI Engineer', 'AI Developer', 'AI Architect', 'AI Research', 'AI Scientist']
    ml_ops_keywords = ['MLOps', 'ML Infrastructure', 'ML Operations', 'ML Software', 'ML Manager', 'ML Modeler']
    data_engineering_keywords = ['Data Engineer', 'Data Integration', 'ETL', 'Big Data', 'Cloud Data Engineer', 'Data Infrastructure', 'Software Data Engineer']
    data_architecture_keywords = ['Data Architect', 'Data Modeler', 'Data Strategist', 'Data Strategy', 'Cloud Data Architect', 'AWS Data Architect']
    bi_visualization_keywords = ['BI Developer', 'Business Intelligence Engineer', 'BI Analyst', 'Data Visualization', 'BI Data Analyst']
    data_quality_ops_keywords = ['Data Quality', 'Data Operations']
    data_product_mgmt_keywords = ['Data Product Manager', 'Data Manager', 'Data Lead', 'Data Science Manager', 'Data Analytics Manager', 'Manager Data Management', 'Data Developer', 'Data Science Practitioner']
    analytics_engineering_keywords = ['Analytics Engineer', 'Data Analytics Specialist', 'Data Analytics Consultant', 'Data Analytics Engineer', 'Data Analytics Lead']
    leadership_executive_keywords = ['Data Science Director', 'Head of Data', 'Director of Data Science', 'Head of ML', 'Head of Data Science', 'Managing Director Data Science', 'Lead', 'Principal']
    senior_principal_roles_keywords = ['Staff', 'Principal']
    cloud_database_keywords = ['Cloud Database Engineer', 'BI Data Engineer']
    data_management_analysis_keywords = ['Data Management Analyst', 'Data Specialist', 'Business Intelligence Manager', 'Business Intelligence Developer', 'Business Intelligence Specialist']
    
       # Check and assign categories based on keywords
    if any(keyword in job_title for keyword in core_data_analysis_keywords):
        return 'Core Data Analysis'
    elif any(keyword in job_title for keyword in advanced_data_science_keywords):
        return 'Advanced Data Science and Research'
    elif any(keyword in job_title for keyword in ml_development_keywords):
        return 'ML Development and Research'
    elif any(keyword in job_title for keyword in ai_specialization_keywords):
        return 'AI Specialization'
    elif any(keyword in job_title for keyword in ml_ops_keywords):
        return 'ML Operations and Infrastructure'
    elif any(keyword in job_title for keyword in data_engineering_keywords):
        return 'Data Engineering and Integration'
    elif any(keyword in job_title for keyword in data_architecture_keywords):
        return 'Data Architecture, Strategy, and Modeling'
    elif any(keyword in job_title for keyword in bi_visualization_keywords):
        return 'BI and Visualization'
    elif any(keyword in job_title for keyword in data_quality_ops_keywords):
        return 'Data Quality and Operations Management'
    elif any(keyword in job_title for keyword in data_product_mgmt_keywords):
        return 'Data Product and Project Management'
    elif any(keyword in job_title for keyword in analytics_engineering_keywords):
        return 'Analytics Engineering and Consulting'
    elif any(keyword in job_title for keyword in leadership_executive_keywords):
        return 'Leadership and Executive Roles'
    elif any(keyword in job_title for keyword in senior_principal_roles_keywords):
        return 'Senior and Principal Roles'
    elif any(keyword in job_title for keyword in cloud_database_keywords):
        return 'Cloud and Database Specialization'
    elif any(keyword in job_title for keyword in data_management_analysis_keywords):
        return 'Data Management and Analysis'
    else:
        return 'Other'

# Applying the function to the job_title column
df['job_category'] = df['job_title'].apply(group_job_titles)

In [ ]:
# 2.3 - check the function
df['job_category'].value_counts()

In [ ]:
#2.3 - check for null values
df.info()

We were able to categorize 94% of the values. The remaining 6% are categorized under 'other'. Depending on the analysis, these values can be removed or left. For this excercise, I will leave them in.**

In [ ]:
# 2.4 - Distribution of Salaries
plt.figure(figsize=(10, 6))
sns.histplot(df['salary'])
plt.title('Distribution of Salaries')
plt.xlabel('Salary (USD)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# 2.4 - Boxplot of salaries by job category
plt.figure(figsize=(15, 7))
sns.boxplot(x='salary', y='job_category', data=df)
plt.title('Salary by Job Category')
plt.xlabel('Salary (USD)')
plt.ylabel('Job Category')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 2.4 - Barplot of average salary by experience

avg_salary_by_experience = df.groupby('experience_level')['salary'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='experience_level', y='salary', data=avg_salary_by_experience)
plt.title('Average Salary by Experience Level')
plt.xlabel('Experience Level')
plt.ylabel('Average Salary (USD)')
plt.show()

In [ ]:
# 2.4 - Work Settings Distribution
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
df['work_setting'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Work Settings')
plt.ylabel('')